<a href="https://colab.research.google.com/github/aakhterov/ML_projects/blob/master/news_sentiment_analysis/classification_news_fine_tunning_and_predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets transformers nltk huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.0 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from datetime import datetime
from collections import Counter
from nltk.util import bigrams, trigrams
from huggingface_hub import notebook_login
from datasets import Dataset, load_dataset, load_from_disk, concatenate_datasets, ClassLabel
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification, DataCollatorWithPadding, TFAutoModel
from tensorflow.keras.losses import SparseCategoricalCrossentropy, BinaryCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.schedules import PolynomialDecay
from tensorflow.keras.metrics import Accuracy

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
base_path = '/content/drive/MyDrive/Colab Notebooks/'

Mounted at /content/drive


In [ ]:
ds = load_from_disk(base_path + 'Data/news_ds')
ds

Dataset({
    features: ['url', 'datetime', 'author', 'title', 'text', 'provider', 'source'],
    num_rows: 13395
})

In [ ]:
ds.unique('provider')

['BBC',
 'The Times of Israel',
 'Al Jazeera',
 'Al Mayadeen',
 'WAFA News Agency',
 'CNN']

In [ ]:
def get_length(example):
  example['len'] = len(example['text'].split())
  return example

In [ ]:
ds_w_len = ds.map(get_length)

In [ ]:
ds_w_len.set_format("pandas")
ds_w_len[:].groupby(["provider", "source"]).agg(['mean', 'count'])

<ipython-input-8-08d70b6d2378>:2: FutureWarning: ['url', 'datetime', 'author', 'title', 'text'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  ds_w_len[:].groupby(["provider", "source"]).agg(['mean', 'count'])


len      
                                                                   mean count
provider            source                                                   
Al Jazeera          site-live-news                           101.171975  3297
Al Mayadeen         site-articles                           1196.256757    74
BBC                 site-live-news                           129.027295   806
                    site-tag(News)-search(israel)            569.427350   117
                    site-tag(News)-search(israel-gaza war)   537.027778    72
CNN                 site-live-news                           197.663165  1428
The Times of Israel site-live-news                           127.901383  6581
WAFA News Agency    site-occupation                          158.311765  1020

In [ ]:
ds_bbs_pos = load_dataset("csv", data_files=base_path + 'Data/bbc_news_ds/bbc_news_ds_pos.csv')['train']
ds_bbs_neg = load_dataset("csv", data_files=base_path + 'Data/bbc_news_ds/bbc_news_ds_neg.csv')['train']

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
# all_posts = "\n".join(ds.filter(lambda x: x["provider"] == "WAFA News Agency")["text"])
# c = Counter(list(bigrams(all_posts.split())))
# c.most_common(100)

In [ ]:
def keywords_filter(example, keywords):
  for keyword in keywords:
    if keyword in example['text'].lower():
      return True
  return False

In [ ]:
keywords_neg_min = ["occup", "crimes", "aggression",  "genocid", "sanction israel", "fighters", "resistance", "israeli regime", "ethnic cleansing"]

In [ ]:
keywords_neg_ext = keywords_neg_min + ["injured", "hundreds of civilians"]

In [ ]:
keywords_pos = ["terror group", "terror", "israel defense force says", "idf says", "daniel hagari", "israeli army spokesperson says"]

In [ ]:
ds_toi = ds.filter(lambda x: x["provider"] == 'The Times of Israel' and not keywords_filter(x, keywords_neg_min))
ds_toi

Dataset({
    features: ['url', 'datetime', 'author', 'title', 'text', 'provider', 'source'],
    num_rows: 6054
})

In [ ]:
ds_aj = ds.filter(lambda x: x["provider"] == 'Al Jazeera' and not keywords_filter(x, keywords_pos))
ds_aj

Dataset({
    features: ['url', 'datetime', 'author', 'title', 'text', 'provider', 'source'],
    num_rows: 3153
})

In [ ]:
ds_wafa = ds.filter(lambda x: x["provider"] == "WAFA News Agency")
ds_wafa

Dataset({
    features: ['url', 'datetime', 'author', 'title', 'text', 'provider', 'source'],
    num_rows: 1020
})

In [ ]:
def split_text(example):
  parts_number = 6
  splitting = example["text"].split()
  words_per_part = int(len(splitting) / parts_number)
  examples = {}
  for i in range(parts_number):
    for k, v in example.items():
      examples[k] = examples.get(k, [])
      if k == 'text':
        examples[k].append(" ".join(splitting[i*words_per_part:(i+1)*words_per_part]))
      else:
        examples[k].append(v)

  return examples

In [ ]:
ds_am = ds.filter(lambda x: x["provider"] == 'Al Mayadeen').map(split_text)
ds_am.set_format("pandas")
df = ds_am[:].explode(ds_am.column_names, ignore_index=True)
ds_am = Dataset.from_pandas(df)
ds_am

Dataset({
    features: ['url', 'datetime', 'author', 'title', 'text', 'provider', 'source'],
    num_rows: 444
})

In [ ]:
ds_am_new_w_len = ds_am.map(get_length)
ds_am_new_w_len.set_format("pandas")
ds_am_new_w_len[:].groupby(["provider", "source"]).agg(['mean', 'count'])

Map:   0%|          | 0/444 [00:00<?, ? examples/s]

<ipython-input-19-c3b093d9d649>:3: FutureWarning: ['url', 'datetime', 'author', 'title', 'text'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  ds_am_new_w_len[:].groupby(["provider", "source"]).agg(['mean', 'count'])


len      
                                 mean count
provider    source                         
Al Mayadeen site-articles  199.013514   444

In [ ]:
# ds_bbc_neg_new = ds.filter(lambda x: x["provider"] == 'BBC' and x["source"] == 'site-live-news' and keywords_filter(x, keywords_neg_min))
# ds_bbc_neg_new.to_csv(base_path + 'Data/bbc_news_ds/bbc_news_ds_neg.csv')

In [ ]:
# ds_bbc_pos_new = ds.filter(lambda x: x["provider"] == 'BBC' and x["source"] == 'site-live-news' and keywords_filter(x, keywords_pos_min))
# ds_bbc_pos_new.to_csv(base_path + 'Data/bbc_news_ds/bbc_news_ds_pos.csv')

In [ ]:
# ds_cnn_neg_new = ds.filter(lambda x: x["provider"] == 'CNN' and keywords_filter(x, keywords_neg_min))
# ds_cnn_neg_new.to_csv(base_path + 'Data/cnn_news_ds/cnn_news_ds_neg.csv')

In [ ]:
# ds_cnn_pos_new = ds.filter(lambda x: x["provider"] == 'CNN' and keywords_filter(x, keywords_pos_min))
# ds_cnn_pos_new.to_csv(base_path + 'Data/cnn_news_ds/cnn_news_ds_pos.csv')

In [ ]:
# ds_toi_neg_new = ds.filter(lambda x: x["provider"] == 'The Times of Israel' and keywords_filter(x, keywords_neg_min))
# ds_toi_neg_new.to_csv(base_path + 'Data/toi_news_ds/toi_news_ds_neg.csv')

In [ ]:
# c = Counter(trigrams(("\n".join(ds.filter(lambda x: x["provider"] == 'The Times of Israel')['text'])).split()))

In [ ]:
# c.most_common(100)

In [ ]:
# ds_aj_new = ds.filter(lambda x: x["provider"] == 'Al Jazeera' and keywords_filter(x, keywords_neg_ext))
# ds_aj_new

In [ ]:
def make_neg_labels(example):
  example['label'] = 0
  return example

def make_pos_labels(example):
  example['label'] = 1
  return example

In [ ]:
ds_neg = concatenate_datasets([ds_am, ds_aj, ds_wafa, ds_bbs_neg]).map(make_neg_labels)
ds_neg

Map:   0%|          | 0/4700 [00:00<?, ? examples/s]

Dataset({
    features: ['url', 'datetime', 'author', 'title', 'text', 'provider', 'source', 'label'],
    num_rows: 4700
})

In [ ]:
ds_pos = concatenate_datasets([ds_toi, ds_bbs_pos]).map(make_pos_labels)
ds_pos

Map:   0%|          | 0/6117 [00:00<?, ? examples/s]

Dataset({
    features: ['url', 'datetime', 'author', 'title', 'text', 'provider', 'source', 'label'],
    num_rows: 6117
})

In [ ]:
ds_classification = concatenate_datasets([ds_neg, ds_pos])
ds_classification

Dataset({
    features: ['url', 'datetime', 'author', 'title', 'text', 'provider', 'source', 'label'],
    num_rows: 10817
})

In [ ]:
# providers = ['The Times of Israel', 'Al Jazeera']
# ds_classification = ds.filter(lambda x: x['provider'] in providers)
# ds_classification

In [ ]:
def merge_title_with_text(example):
  example['body'] = (example['title'] + '\n' + example['text']) if example['title'] else example['text']
  return example

In [ ]:
ds_classification = ds_classification.map(merge_title_with_text)
ds_classification

Map:   0%|          | 0/10817 [00:00<?, ? examples/s]

Dataset({
    features: ['url', 'datetime', 'author', 'title', 'text', 'provider', 'source', 'label', 'body'],
    num_rows: 10817
})

In [ ]:
ds_classification = ds_classification.select_columns(['body', 'label'])
ds_classification

Dataset({
    features: ['body', 'label'],
    num_rows: 10817
})

In [ ]:
ds_negative = ds_classification.filter(lambda x: x["label"] == 0)
negotive_count = len(ds_negative)

ds_pos_balanced = ds_classification.filter(lambda x: x["label"] == 1).shuffle().select(range(negotive_count))

ds_balanced = concatenate_datasets([ds_pos_balanced, ds_negative])

print(f"Positive samples count: {len(ds_balanced.filter(lambda x: x['label'] == 1))}")
print(f"Negative samples count: {len(ds_balanced.filter(lambda x: x['label'] == 0))}")

Filter:   0%|          | 0/10817 [00:00<?, ? examples/s]

Filter:   0%|          | 0/10817 [00:00<?, ? examples/s]

Filter:   0%|          | 0/9400 [00:00<?, ? examples/s]

Positive samples count: 4700


Filter:   0%|          | 0/9400 [00:00<?, ? examples/s]

Negative samples count: 4700


In [ ]:
ds_splitted = ds_balanced.train_test_split(train_size=.8)
test_and_val_ds = ds_splitted['test'].train_test_split(train_size=0.75)
ds_splitted['test'] = test_and_val_ds['train']
ds_splitted['validation'] = test_and_val_ds['test']
ds_splitted

DatasetDict({
    train: Dataset({
        features: ['body', 'label'],
        num_rows: 7520
    })
    test: Dataset({
        features: ['body', 'label'],
        num_rows: 1410
    })
    validation: Dataset({
        features: ['body', 'label'],
        num_rows: 470
    })
})

In [ ]:
ds_splitted['train'].features

{'body': Value(dtype='string', id=None),
 'label': Value(dtype='int64', id=None)}

In [ ]:
new_features = ds_splitted['train'].features.copy()
new_features['label'] = ClassLabel(names=['neg', 'pos'])
ds_splitted = ds_splitted.cast(new_features)

Casting the dataset:   0%|          | 0/7520 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/1410 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/470 [00:00<?, ? examples/s]

In [ ]:
# checkpoint = "roberta-base"
# checkpoint = "bert-base-uncased"
checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
tokenizer(ds_splitted['train']['body'][0])

NameError: ignored

In [ ]:
def make_tokenize(example):
  return tokenizer(example['body'], truncation=True)

In [ ]:
ds_tokenized = ds_splitted.map(make_tokenize, batched=True)

Map:   0%|          | 0/7520 [00:00<?, ? examples/s]

Map:   0%|          | 0/1410 [00:00<?, ? examples/s]

Map:   0%|          | 0/470 [00:00<?, ? examples/s]

In [ ]:
ds_tokenized

DatasetDict({
    train: Dataset({
        features: ['body', 'label', 'input_ids', 'attention_mask'],
        num_rows: 7520
    })
    test: Dataset({
        features: ['body', 'label', 'input_ids', 'attention_mask'],
        num_rows: 1410
    })
    validation: Dataset({
        features: ['body', 'label', 'input_ids', 'attention_mask'],
        num_rows: 470
    })
})

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors='tf')

In [ ]:
batch_size = 8
num_epochs = 5

In [ ]:
tf_train_dataset = ds_tokenized["train"].to_tf_dataset(
    columns=["attention_mask", "input_ids"],
    label_cols=["label"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=batch_size,
)

tf_validation_dataset = ds_tokenized["validation"].to_tf_dataset(
    columns=["attention_mask", "input_ids"],
    label_cols=["label"],
    shuffle=False,
    collate_fn=data_collator,
    batch_size=batch_size,
)

tf_test_dataset = ds_tokenized["test"].to_tf_dataset(
    columns=["attention_mask", "input_ids"],
    label_cols=["label"],
    shuffle=False,
    collate_fn=data_collator,
    batch_size=batch_size,
)

/usr/local/lib/python3.10/dist-packages/datasets/arrow_dataset.py:399: FutureWarning: The output of `to_tf_dataset` will change when a passing single element list for `labels` or `columns` in the next datasets version. To return a tuple structure rather than dict, pass a single string.
Old behaviour: columns=['a'], labels=['labels'] -> (tf.Tensor, tf.Tensor)  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor)  
New behaviour: columns=['a'],labels=['labels'] -> ({'a': tf.Tensor}, {'labels': tf.Tensor})  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor) 
  warnings.warn(
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [ ]:
model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_layer_norm.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

In [ ]:
num_train_steps = len(tf_train_dataset) * num_epochs

lr_scheduler = PolynomialDecay(initial_learning_rate=5e-5,
                               end_learning_rate=0.0,
                               decay_steps=num_train_steps)

In [ ]:
model.compile(
    optimizer=Adam(learning_rate=lr_scheduler),
    loss=SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)

In [ ]:
model.summary()

Model: "tf_distil_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 distilbert (TFDistilBertMa  multiple                  66362880  
 inLayer)                                                        
                                                                 
 pre_classifier (Dense)      multiple                  590592    
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
 dropout_19 (Dropout)        multiple                  0         
                                                                 
Total params: 66955010 (255.41 MB)
Trainable params: 66955010 (255.41 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
model.fit(
    tf_train_dataset,
    validation_data=tf_validation_dataset,
    epochs=num_epochs
)

Epoch 1/5
940/940 [==============================] - 348s 343ms/step - loss: 0.1159 - accuracy: 0.9581 - val_loss: 0.0671 - val_accuracy: 0.9787
Epoch 2/5
940/940 [==============================] - 297s 316ms/step - loss: 0.0356 - accuracy: 0.9890 - val_loss: 0.0248 - val_accuracy: 0.9936
Epoch 3/5
940/940 [==============================] - 300s 319ms/step - loss: 0.0113 - accuracy: 0.9960 - val_loss: 0.0237 - val_accuracy: 0.9915
Epoch 4/5
940/940 [==============================] - 296s 315ms/step - loss: 0.0056 - accuracy: 0.9980 - val_loss: 0.0235 - val_accuracy: 0.9957
Epoch 5/5
940/940 [==============================] - 301s 320ms/step - loss: 8.6653e-04 - accuracy: 0.9999 - val_loss: 0.0260 - val_accuracy: 0.9957


In [ ]:
model.save_weights(base_path + f'checkpoints/{checkpoint}/{checkpoint}')

In [ ]:
model.load_weights(base_path + f'checkpoints/{checkpoint}/{checkpoint}')

In [ ]:
# model.save(base_path + f'models/{checkpoint}/{checkpoint}')

In [ ]:
def get_accuracy(y_true, logits_pred):
  class_preds = np.argmax(logits_pred, axis=1)
  acc = Accuracy()
  acc.update_state(y_true, class_preds)
  return acc.result().numpy()

In [ ]:
def get_samples_by_class(ds, class_predictions, current_class, samples_count):
  cond = class_predictions == current_class
  indexes = np.where(cond == 1)[0]
  filtered_ds = ds.filter(lambda x, idx: idx in indexes, with_indices=True)
  return filtered_ds.shuffle().select(range(samples_count))

In [ ]:
preds = model.predict(tf_validation_dataset)["logits"]
get_accuracy(ds_tokenized["validation"]['label'], preds)

59/59 [==============================] - 8s 118ms/step


0.9957447

In [ ]:
preds = model.predict(tf_test_dataset)["logits"]
get_accuracy(ds_tokenized["test"]['label'], preds)

177/177 [==============================] - 19s 107ms/step


0.9879433

In [ ]:
class_preds = np.argmax(preds, axis=1)
counts = 1
for i in range(len(ds_tokenized["test"])):
  if ds_tokenized["test"]['label'][i] != class_preds[i]:
    print(f"== {counts} {10*'='}")
    print(f"True label: {ds_tokenized['test']['label'][i]}")
    print(f"Predicted label: {class_preds[i]}")
    print(f"Predicted vector: {preds[i]}, {tf.math.softmax(preds[i])}")
    print(f"Body: {ds_tokenized['test']['body'][i]}")
    counts += 1

== 1 ==========
True label: 1
Predicted label: 0
Predicted vector: [ 0.4558785  -0.26043764], [0.67179525 0.32820472]
Body: Biden praises Netanyahu, Sissi on Gaza humanitarian aid deal: They ‘stepped up’
US President Joe Biden praised Prime Minister Benjamin Netanyahu and Egyptian President Abdel Fattah al-Sissi for an agreement to let humanitarian aid in to the Gaza Strip through the Rafah crossing from Egypt.
Speaking to reporters at the Ramstein Air Base in Germany following a brief solidarity visit to Israel on Wednesday, Biden said Sissi agreed to open the crossing and to let in an initial group of 20 trucks with humanitarian aid, and possibly more at a later time.
“Sissi deserves some real credit because he was very accommodating,” Biden said, adding that the Egyptian president was “fair” and “very cooperative.”
Israel stopped all entry of food, water, medicine and fuel to Gaza following Hamas’s brutal onslaught on October 7 when some 2,500 terrorists burst across the border into

In [ ]:
ds_bbc_tokenized = ds.filter(lambda x: x['provider'] == 'BBC' and x['source'] == 'site-live-news')\
                     .map(merge_title_with_text)\
                     .select_columns(['body'])\
                     .map(make_tokenize, batched=True)
ds_bbc_tokenized

Dataset({
    features: ['body', 'input_ids', 'attention_mask'],
    num_rows: 806
})

In [ ]:
tf_bbc_dataset = ds_bbc_tokenized.to_tf_dataset(
    columns=["attention_mask", "input_ids"],
    shuffle=False,
    collate_fn=data_collator,
    batch_size=batch_size,
)

In [ ]:
preds_bbc = model.predict(tf_bbc_dataset)["logits"]

101/101 [==============================] - 11s 95ms/step


In [ ]:
softmax = tf.math.softmax(preds_bbc, axis=1)
thr = 0.3
class_preds_bbc = []
for v in softmax.numpy():
  if v[1] - v[0] >= thr:
    class_preds_bbc.append(1)
  elif v[0] - v[1] >= thr:
    class_preds_bbc.append(-1)
  else:
    class_preds_bbc.append(0)
class_preds_bbc = np.array(class_preds_bbc)

In [ ]:
total_pos = np.sum(class_preds_bbc == 1)
total_neg = np.sum(class_preds_bbc == -1)
total_nrl = np.sum(class_preds_bbc == 0)

print(f'Total number of samples: {len(class_preds_bbc)}')
print(f'Predicted as positive: {total_pos}/{total_pos/len(class_preds_bbc):.0%}')
print(f'Predicted as neutral: {total_nrl}/{total_nrl/len(class_preds_bbc):.0%}')
print(f'Predicted as negative: {total_neg}/{total_neg/len(class_preds_bbc):.0%}')

Total number of samples: 806
Predicted as positive: 165/20%
Predicted as neutral: 32/4%
Predicted as negative: 609/76%


In [ ]:
get_samples_by_class(ds_bbc_tokenized, class_preds_bbc, -1, 3)['body']

["The latest developments from Al-Shifa\nIt's gone 18:00 in Gaza - and 16:00 here in our London newsroom. \nIt's still hard to know exactly what is going on inside Al-Shifa hospital in Gaza City, but here's a look at the reported developments over the last few hours: ",
 '\'Those who are staying in Al-Shifa hospital already decided they are dead\'\nDr. Ahmed El Mokhallalati, a senior plastic surgeon at the Al-Shifa hospital in Gaza City says that Israeli forces have "bombed continuously at Al-Shifa hospital and the surrounding areas" since last night, according to Reuters. \nThe hospital building has been destroyed, he says, and is now "totally out of service".\nCivilians who were sheltering in the building and many patients requiring urgent treatment have since fled - some of whom were reportedly shot at, according to the medical aid organisation Doctors Without Borders\nSeveral medical personnel have also been forced to flee, leaving the hospital with "no more 20 or 15 percent of the

In [ ]:
get_samples_by_class(ds_bbc_tokenized, class_preds_bbc, 0, 3)['body']

["Israel says it took over Hamas stronghold in northern Gaza Strip\nThe Israel Defense Forces has just given its morning update. It says:\nAlthough the IDF didn't mention it, we're getting pictures of the aftermath of an apparent air strike in Khan Younis in the southern Gaza Strip - we'll bring you those soon.\nAs a reminder, the Hamas-run health ministry in Gaza says more more than 10,000 people have been killed since Israel began its operation on 7 October, following the Hamas attacks.",
 "'We walked into a wasteland' - BBC's Jeremy Bowen in Gaza with Israeli forces\nThe BBC's international editor Jeremy Bowen has travelled with the Israel Defense Forces (IDF) into Gaza.\nWhile the BBC had editorial control of the report, the section with the IDF has been viewed by them.\nThe IDF showed the BBC what they said was a Hamas weapons factory, below a home in which children lived.",
 'Hundreds of Hamas supporters attend Lebanon rally\nThere are almost 500,000 Palestinians or descendants o

In [ ]:
get_samples_by_class(ds_bbc_tokenized, class_preds_bbc, 1, 3)['body']

['We will free Gaza from Hamas - Hagari\nAs he concludes his remarks, Hagari says: "This is not the last hospital like this in Gaza and the world should know that". \n"Whoever gives money to this hospital... is giving Hamas help," he adds. \n"We will free our hostages from Gaza and free Gaza from Hamas for the sake of the people of Israel and for the people of Gaza as well."',
 'Hamas officials say Gaza death toll passes 12,000\nHamas officials say more than 12,000 Palestinians - including 5,000 children - are now known to have been killed by Israeli military action in the Gaza Strip since 7\nOctober. \nThe Palestinian Information Center, which\ncarries official Hamas statements, also says more than 3,570 people –\nincluding 1,800 children – are missing.',
 'UN refugee agency says it won\'t be able to pick up aid from tomorrow\nThe UN\'s Palestinian refugee agency (Unrwa) - which runs the largest humanitarian operation in Gaza - has told the BBC that, from tomorrow, it will be unable t

In [ ]:
ds_cnn_tokenized = ds.filter(lambda x: x['provider'] == 'CNN')\
                     .map(merge_title_with_text)\
                     .select_columns(['body'])\
                     .map(make_tokenize, batched=True)
ds_cnn_tokenized

Dataset({
    features: ['body', 'input_ids', 'attention_mask'],
    num_rows: 1428
})

In [ ]:
tf_cnn_dataset = ds_cnn_tokenized.to_tf_dataset(
    columns=["attention_mask", "input_ids"],
    shuffle=False,
    collate_fn=data_collator,
    batch_size=batch_size,
)

In [ ]:
preds_cnn = model.predict(tf_cnn_dataset)["logits"]

179/179 [==============================] - 25s 139ms/step


In [ ]:
softmax = tf.math.softmax(preds_cnn, axis=1)
thr = 0.3
class_preds_cnn = []
for v in softmax.numpy():
  if v[1] - v[0] >= thr:
    class_preds_cnn.append(1)
  elif v[0] - v[1] >= thr:
    class_preds_cnn.append(-1)
  else:
    class_preds_cnn.append(0)
class_preds_cnn = np.array(class_preds_cnn)

In [ ]:
total_pos = np.sum(class_preds_cnn == 1)
total_neg = np.sum(class_preds_cnn == -1)
total_nrl = np.sum(class_preds_cnn == 0)

print(f'Total number of samples: {len(class_preds_cnn)}')
print(f'Predicted as positive: {total_pos}/{total_pos/len(class_preds_cnn):.0%}')
print(f'Predicted as neutral: {total_nrl}/{total_nrl/len(class_preds_cnn):.0%}')
print(f'Predicted as negative: {total_neg}/{total_neg/len(class_preds_cnn):.0%}')

Total number of samples: 1428
Predicted as positive: 397/28%
Predicted as neutral: 110/8%
Predicted as negative: 921/64%


In [ ]:
get_samples_by_class(ds_cnn_tokenized, class_preds_cnn, -1, 3)['body']

Filter:   0%|          | 0/1428 [00:00<?, ? examples/s]

['No announcements on the Rafah crossing opening for civilians, US State Department says \nUS State Department spokesperson Matt Miller said Tuesday that he had no announcements to make about the opening of the Rafah crossing for civilians but noted that “we have made good progress, even in the past few hours.”\nMiller said the department was working intensively on the issue, with Ambassador David Satterfield working the matter on the ground.\nUS Secretary of State Antony Blinken spoke with his Qatari counterpart Monday about pressing Hamas to allow Americans and other foreign nationals to leave, Miller said.\nThe official said that once an agreement is reached, his understanding is it is not a process that would occur instantly. He encouraged all Americans in Gaza who wish to leave to register with the State Department.\nBlinken told lawmakers earlier Tuesday, that approximately 400 American citizens and their family members – about 1,000 people total – are stuck in Gaza and are seeki

In [ ]:
get_samples_by_class(ds_cnn_tokenized, class_preds_cnn, 0, 3)['body']

Filter:   0%|          | 0/1428 [00:00<?, ? examples/s]

['Israeli troops have advanced over two miles into northern Gaza, video suggests \nIsraeli troops appear to have advanced over two miles into Gaza, according to a CNN analysis of video published by an Israeli media outlet.\xa0\nThe troops in the video, taken on Saturday, are seen putting an Israeli flag on a Gaza resort hotel\'s roof.\xa0CNN geolocated the video to an area just over two miles from the Gaza-Israeli border.\n"Soldiers of the 52 Battalion of the 401 Brigade are waving the Israeli flag in the heart of Gaza, by the beach," a soldier is heard saying in the video, taken several miles north of central Gaza City.\xa0"We will not forgive nor forget, and we’ll not stop until the victory."\nThe video is one of the first glimpses into where Israeli ground forces have been, and what they\'ve been doing, during the expanded ground operations in Gaza.\xa0\nA communications blackout in Gaza has significantly hampered the flow of information out of it, though providers said service was 

In [ ]:
get_samples_by_class(ds_cnn_tokenized, class_preds_cnn, 1, 3)['body']

Filter:   0%|          | 0/1428 [00:00<?, ? examples/s]

['Blinken says it\'s shocking that October 7 Hamas attack "has receded so quickly in the memories of so many"\nUS Secretary of State Antony Blinken called it "striking" and "shocking" that the brutality of the October 7 Hamas attack on Israel "has receded so quickly in the memories of so many" during a news conference in Tel Aviv on Friday. \nBlinken said today he viewed additional images and footage from the attack collected by the Israeli government.\nBlinken, who appeared visibly shaken, described an example of additional video Israeli officials showed him today of the Hamas attack.\nThe top US diplomat expressed sympathy for the plight of Palestinians civilians as well, saying he sees his own children when he sees images of “Palestinian children, young boys and girls, pulled from the wreckage and buildings.”\xa0\nBlinken said that the US is still pressing Israel to minimize civilian casualties.\n“We’ve provided Israel advice that only the best of friends can offer on how to minimiz

In [ ]:
def predict(tokenizer, model, text):
  text_tokenized = tokenizer(text, truncation=True, return_tensors="tf")
  output = model(**text_tokenized)
  result = tf.math.softmax(output["logits"]).numpy()
  return result

In [ ]:
text = '''
Earlier this morning, an official for UNRWA - the UN's agency for Palestinian refugees - spoke to the BBC from Rafah in south Gaza. He described a "desperate situation" for locals.
Tom White told BBC Radio 4's Today programme that a ceasefire was crucial. "Here in Rafah we have hundreds of thousands of people who are living in the open," he explained. "There is a lack of water. Everyone in the street is asking for flour to feed their children.
"Our shelters have well over 7,000 people. There are hundreds using the one toilet for example.
"If the bombs aren't going to kill them, it is the disease, or for those living out on the streets, it'll be the exposure."
Describing his trip to a UN distribution centre in Rafah on Friday, White said: "All you could hear was air strikes going into the city." He also told the BBC a guesthouse he shared with colleagues was hit last night.
'''

In [ ]:
text = '''
KHAN YUNIS, Sunday, December 10, 2023 (WAFA) – At least 10 civilians were killed, mostly children, and dozens more were wounded early this morning as Israeli warplanes bombed a residential house in Khan Yunis, south of the Gaza Strip, as the Israeli aggression on the enclave enters its 65th day in a row.
Medical sources confirmed the death toll resulting from the Israeli airstrike and reported further injuries, along with several individuals missing under the desbris, following the Israeli bombardment which targeted a house belonging to the Abdulwahab family west of Khan Yunis.
Israeli artillery also shelled the vicinity of the European Hospital in the city, which has been under a tight Israeli military siege and ground invasion for over a week now.
In the central Gaza Strip, medical sources at the Al-Aqsa Martyrs Hospital in the city of Deir al-Balah confirmed the arrival of several fatalities and wounded following an Israeli airstrike which targeted a residential building in the city.
Intense Israeli air raids were reported also in the nearby refugee camps of Nuseirat, Maghazi, and the town of Al-Zawaida in the central region of the strip.
Additionally, Israeli airstrikes targeted areas in the neighborhoods of Tuffah and Shujaeya east of Gaza City, as well as various locations in northern Gaza.
Furthermore, Israeli occupation forces renewed their artillery bombardment on the Jabalia refugee camp in northern Gaza, which also has been under a ground invasion for weeks.
The ongoing Israeli aggression has brought immense suffering to the civilian Palestinian population in the Gaza Strip, with casualties, particularly among innocent children and healthcare workers, mounting around the clock.
'''

In [ ]:
text = '''
Several thousand people demonstrate against antisemitism in Berlin as Germany grapples with a large increase in anti-Jewish incidents following Hamas’s assault on Israel two months ago.
Police estimate that around 3,200 people gathered in the rain in the German capital, while organizers put the figure at 10,000, German news agency dpa reports. Participants in the protest, titled “Never again is now,” march to the Brandenburg Gate.
Germany’s labor minister, Hubertus Heil, says that many decent people are too quiet on the growing antisemitic sentiment in the country: “We don’t need a decent, silent majority — we need a clear and loud majority that stands up now, and not later,” he says.
The event had wide support, with the speaker of the German parliament and Berlin’s mayor among its backers.
'''

In [ ]:
predict(tokenizer, model, text)

array([[6.569553e-05, 9.999343e-01]], dtype=float32)

In [ ]:
# np.array(ds_cnn_tokenized['body'])[class_preds == -1]

In [ ]:
# model.load_weights(base_path + f'checkpoints/{checkpoint}')

In [ ]:
embedding_model = 'sentence-transformers/multi-qa-mpnet-base-dot-v1'
tokenizer = AutoTokenizer.from_pretrained(embedding_model)
model_ebd = TFAutoModel.from_pretrained(embedding_model, from_pt=True)

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFMPNetModel: ['embeddings.position_ids']
- This IS expected if you are initializing TFMPNetModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFMPNetModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFMPNetModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMPNetModel for predictions without further training.


In [ ]:
def cls_pooling(model_output):
    return model_output.last_hidden_state[:, 0]

In [ ]:
def get_embeddings(text_list):
  encoded_input = tokenizer(text_list, padding=True, truncation=True, return_tensors="tf")
  encoded_input = {k: v for k, v in encoded_input.items()}
  model_output = model_ebd(**encoded_input)
  return cls_pooling(model_output)

In [ ]:
ds_embedded = ds_splitted.map(lambda x: {'embedding': get_embeddings(x['body']).numpy()[0]})

/usr/local/lib/python3.10/dist-packages/transformers/generation/tf_utils.py:465: UserWarning: `seed_generator` is deprecated and will be removed in a future version.
  warnings.warn("`seed_generator` is deprecated and will be removed in a future version.", UserWarning)


Map:   0%|          | 0/3672 [00:00<?, ? examples/s]

Map:   0%|          | 0/688 [00:00<?, ? examples/s]

Map:   0%|          | 0/230 [00:00<?, ? examples/s]

In [ ]:
ds_embedded.save_to_disk(base_path + '/Data/news_ds_embedded')

NameError: ignored

In [ ]:
ds_embedded = load_from_disk(base_path + '/Data/news_ds_embedded')

In [ ]:
ds_embedded

DatasetDict({
    train: Dataset({
        features: ['body', 'label', 'embedding'],
        num_rows: 3672
    })
    test: Dataset({
        features: ['body', 'label', 'embedding'],
        num_rows: 688
    })
    validation: Dataset({
        features: ['body', 'label', 'embedding'],
        num_rows: 230
    })
})

In [ ]:
pos_ds_embedded = ds_embedded['train'].filter(lambda x: x['label']==1)
neg_ds_embedded = ds_embedded['train'].filter(lambda x: x['label']==0)

In [ ]:
pos_ds_embedded.add_faiss_index(column='embedding')
neg_ds_embedded.add_faiss_index(column='embedding')

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Dataset({
    features: ['body', 'label', 'embedding'],
    num_rows: 1838
})

In [ ]:
def get_nearest_opposite_example(example):
  if example['label'] == 1:
    scores, samples = neg_ds_embedded.get_nearest_examples("embedding", np.array(example['embedding']).astype(np.float32), k=1)
  else:
    scores, samples = pos_ds_embedded.get_nearest_examples("embedding", np.array(example['embedding']).astype(np.float32), k=1)
  example['score'] = scores[0]
  example['nearest_opposite_post'] = samples['body']
  return example

In [ ]:
ds_train_extended = ds_embedded['train']\
                    .map(get_nearest_opposite_example)\
                    .sort('score')

Map:   0%|          | 0/3672 [00:00<?, ? examples/s]

In [ ]:
index = 6
ds_train_extended[index]['score'], ds_train_extended[index]['label'], ds_train_extended[index]['body'], ds_train_extended[index]['nearest_opposite_post']

(3.8930892944335938,
 1,
 'Israel dismisses UN rights chief request for access to country\nIsrael dismisses a request from the UN rights chief to access the country after he called for an international probe into the Israel-Hamas war and appeared to equate Israeli actions to those of the Hamas terror group.\n“Israel is not aware of any added benefit of the high commissioner’s visit at this time,” the country’s mission to the UN in Geneva tells AFP, when asked about Volker Turk’s request to be permitted to visit.',
 ['Israel dismisses UN rights chief’s request for access to country\n“Israel is not aware of any added benefit of the high commissioner’s visit at this time,” the Israeli mission to the UN in Geneva told AFP, when asked about Volker Turk’s request to visit.'])

In [ ]:
score_threhold = 5.0

In [ ]:
ds_with_similar_samples = ds_train_extended.filter(lambda x: x['score'] < score_threhold)

Filter:   0%|          | 0/3672 [00:00<?, ? examples/s]

In [ ]:
ds_splitted['train'] = ds_splitted['train'].filter(lambda x: x['body'] not in ds_with_similar_samples['body'])

Filter:   0%|          | 0/3672 [00:00<?, ? examples/s]

In [ ]:
ds_embedded["train"].add_faiss_index(column='embedding')

  0%|          | 0/4 [00:00<?, ?it/s]

Dataset({
    features: ['body', 'label', 'embedding'],
    num_rows: 3672
})

In [ ]:
txt = '''
Earlier this morning, an official for UNRWA - the UN's agency for Palestinian refugees - spoke to the BBC from Rafah in south Gaza. He described a "desperate situation" for locals.
Tom White told BBC Radio 4's Today programme that a ceasefire was crucial. "Here in Rafah we have hundreds of thousands of people who are living in the open," he explained. "There is a lack of water. Everyone in the street is asking for flour to feed their children.
"Our shelters have well over 7,000 people. There are hundreds using the one toilet for example.
"If the bombs aren't going to kill them, it is the disease, or for those living out on the streets, it'll be the exposure."
Describing his trip to a UN distribution centre in Rafah on Friday, White said: "All you could hear was air strikes going into the city." He also told the BBC a guesthouse he shared with colleagues was hit last night.
'''

In [ ]:
index = 6
query = get_embeddings(txt).numpy()
scores, samples = ds_embedded["train"].get_nearest_examples("embedding", query, k=5)
samples['label'], ds_bbc_tokenized['body'][index]

([0, 1, 0, 0, 1],
 'What the footage from al-Fakhoura school shows\nFootage has emerged online claiming to show the aftermath of a strike on\nthe UNRWA-run al-Fakhoura School in Jabalia – which has been used a shelter for\ndisplaced people. \nBBC Verify has geolocated the footage to the school.\nThe video, which is 1 minute 40 seconds long, begins at the western\ncorner of the school building and tracks through various rooms and stairwells\nas the camera moves down towards the ground floor.\nMany people – including women and children – are seen with severe\ninjuries or lying motionless on the floor in different parts of the building.\nThere are more than 20 such casualties visible in the footage, and around half\nof these are seen in one particular room on the ground floor, which also shows\nsigns of considerable damage.\nUN-run schools have been used as shelters for Gazans in the north. The\nweather conditions evident in the footage match the forecast from today and no\nprevious versi